## Python version of 2.0's Number of Tweezers calculations so we can use them as a function
- Jeff


In [1]:
### Useful functions
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
from get_polarizability import *
import math 

# constants
h = 6.62607015e-34  # Planck's constant in J*s
hbar = h / (2 * np.pi)  # reduced Planck's constant
c = 299792458  # speed of light in m/s
epsilon_0 = 8.854187817e-12  # vacuum permittivity in F/m
a0 = 5.29177210903e-11  # Bohr radius in meters
e = 1.602176634e-19  # elementary charge in Coulombs
me = 9.10938356e-31  # electron mass in kg
kB = 1.380649e-23  # Boltzmann constant in J/K

# Cs information
Cs_D1_wavelength = 894.592959e-9  # Cs D1 line wavelength in meters
Cs_D1_frequency = c / Cs_D1_wavelength  # Cs D1 line frequency in Hz
Cs_D1_gamma = 4.5612e6 # Cs D1 line natural linewidth in Hz
Cs_D1_pi_Isat = 2.4981*10  # Cs D1 line pi-polarized saturation intensity in W/m^2

Cs_D2_wavelength = 852.34727582e-9  # Cs D2 line wavelength in meters
Cs_D2_frequency = c / Cs_D2_wavelength  # Cs D2 line frequency in Hz
Cs_D2_gamma = 5.2227e6 # Cs D2 line natural linewidth in Hz
Cs_D2_pi_Isat = 1.6536*10  # Cs D2 line pi-polarized saturation intensity in W/m^2
Cs_D2_iso_Isat = 2.7059*10  # Cs D2 line isotropic saturation intensity in W/m^2
Cs_D2_sigma_Isat = 1.1023*10  # Cs D2 line sigma-polarized saturation intensity in W/m^2

# Rb information
Rb_D1_wavelength = 794.978850e-9      # Rb D1 line wavelength in meters
Rb_D1_frequency = c / Rb_D1_wavelength  # Rb D1 line frequency in Hz
Rb_D1_gamma = 5.746e6 # Rb D1 line natural linewidth in Hz
Rb_D1_pi_Isat = 4.484*10  # Rb D1 line pi-polarized saturation intensity in W/m^2

Rb_D2_wavelength = 780.241209686e-9  # Rb D2 line wavelength in meters
Rb_D2_frequency = c / Rb_D2_wavelength  # Rb D2 line frequency in Hz
Rb_D2_gamma = 6.065e6 # Rb D2 line natural linewidth in Hz
Rb_D2_pi_Isat = 2.503*10  # Rb D2 line pi-polarized saturation intensity in W/m^2
Rb_D2_iso_Isat = 3.576*10  # Rb D2 line isotropic saturation intensity in W/m^2
Rb_D2_sigma_Isat = 1.669*10  # Rb D2 line sigma-polarized saturation intensity in W/m^2

pol_conversion = 4*np.pi*epsilon_0*(a0**3)

Rb_mass_u = 86.909 # Rb87 in atomic units (u)
Cs_mass_u = 132.905 # Cs133 in atomic units (u)
u_to_kg = 1.66054e-27
Rb_mass = Rb_mass_u*u_to_kg
Cs_mass = Cs_mass_u*u_to_kg

In [14]:
## Experiment parameters
Objective_NA = 0.6
# Trap power
Rb_tweezer_power = 6 # The Sirah Matisse C produces 6W at 809
Cs_tweezer_power = 50 # You can buy 50W of 1064 reasonably
# Trap wavelength
Rb_tweezer_wavelength = 809e-9 # From Keesling thesis
Cs_tweezer_wavelength = 1064e-9 # What 1.0 is using
# Trap freqs
RbRadialTrapFreq = 2*np.pi*0.0987e6 # From Alexander Keesling thesis
CsRadialTrapFreq = 2*np.pi*0.150e6 # From 1.5 most strict trap freq
# Experimental beam waists
Rb_wo = 900e-9 # From Keesling thesis
Cs_wo = 1.14*1064e-9 # From 1.5
# Diffraction limited beam waists
def DL_waist(wavelength, NA):
    # Assume the tweezer is Gaussian, get the 1/e^2 radius
    return wavelength/(np.pi*NA)
Rb_woDL = DL_waist(Rb_tweezer_wavelength, Objective_NA)
Cs_woDL = DL_waist(Cs_tweezer_wavelength, Objective_NA)

In [ ]:
print(pol_Rb(Rb_tweezer_wavelength)/pol_conversion)
print(pol_Cs(Cs_tweezer_wavelength)/pol_conversion)


5961.61
1167.5029999999997


In [15]:
# Assume atoms are in electronic ground state when trapping
def pol_Rb(wavelength):
    return get_polarizability(wavelength*1e9, atom="Rb", orbital="5s") * pol_conversion

def pol_Cs(wavelength):
    return get_polarizability(wavelength*1e9, atom="Cs", orbital="6s") * pol_conversion

def wr_tweezer(pol, intensity, mass, wo):
    return np.sqrt(4*pol*intensity/(2*epsilon_0*c*mass*(wo**2))) 

def Io_from_radial(pol, mass, wo ,wr):
    return (mass*(wr**2)*(wo**2)*epsilon_0*c)/(2*pol)

def P_from_radial(pol, mass, wo, wr):
    return Io_from_radial(pol, mass, wo, wr) * np.pi * (wo**2)/2

def Uo_from_radial(mass, wo, wr):
    return mass*(wr**2)*(wo**2)/(4*kB)


def format_si(x, sig=3):
    prefixes = {
        -24: 'y', -21: 'z', -18: 'a', -15: 'f', -12: 'p',
         -9: 'n',  -6: 'µ',  -3: 'm',   0: '',
          3: 'k',   6: 'M',   9: 'G',  12: 'T'
    }
    if x == 0:
        return f"0 {prefixes[0]}"
    exp = int(math.floor(math.log10(abs(x)) / 3) * 3)
    exp = max(min(exp, 12), -24)
    return f"{x / 10**exp:.{sig}g} {prefixes[exp]}"

Rb_Io_radial = Io_from_radial(pol_Rb(Rb_tweezer_wavelength), Rb_mass, Rb_wo, RbRadialTrapFreq)
Cs_Io_radial = Io_from_radial(pol_Cs(Cs_tweezer_wavelength), Cs_mass, Cs_wo, CsRadialTrapFreq)
Rb_power_per_trap = P_from_radial(pol_Rb(Rb_tweezer_wavelength), Rb_mass, Rb_wo, RbRadialTrapFreq)
Cs_power_per_trap = P_from_radial(pol_Cs(Cs_tweezer_wavelength), Cs_mass, Cs_wo, CsRadialTrapFreq)
Rb_U0 = Uo_from_radial(Rb_mass, Rb_wo, RbRadialTrapFreq)
Cs_U0 = Uo_from_radial(Cs_mass, Cs_wo, CsRadialTrapFreq)

Rb_Io_radial_ideal = Io_from_radial(pol_Rb(Rb_tweezer_wavelength), Rb_mass, Rb_woDL, RbRadialTrapFreq)
Cs_Io_radial_ideal = Io_from_radial(pol_Cs(Cs_tweezer_wavelength), Cs_mass, Cs_woDL, CsRadialTrapFreq)
Rb_power_per_trap_ideal = P_from_radial(pol_Rb(Rb_tweezer_wavelength), Rb_mass, Rb_woDL, RbRadialTrapFreq)
Cs_power_per_trap_ideal = P_from_radial(pol_Cs(Cs_tweezer_wavelength), Cs_mass, Cs_woDL, CsRadialTrapFreq)
Rb_U0_ideal = Uo_from_radial(Rb_mass, Rb_woDL, RbRadialTrapFreq)
Cs_U0_ideal = Uo_from_radial(Cs_mass, Cs_woDL, CsRadialTrapFreq)

## PROBLEM: GET_POL PRODUCES DIFFERENT POLARIZATIONS THAN THE ONES THAT 2.0 IS USING IN THEIR WOLFRAM NOTEBOOK

In [17]:
print(f"Cs Radial Trap Intensity: {format_si(Cs_Io_radial)}W/m^2 or {format_si(Cs_Io_radial/1e4)}W/cm^2")
print(f"Cs Power Per Trap: {format_si(Cs_power_per_trap)}W")
print(f"Cs Trap Depth/Temperature: {format_si(Cs_U0)}K")
print("")
print(f"Cs Radial Trap Intensity w/ Diffraction Limited Beam Waist: {format_si(Cs_Io_radial_ideal)}W/m^2 or {format_si(Cs_Io_radial_ideal/1e7)}W/cm^2")
print(f"Cs Power Per Trap w/ Diffraction Limited Beam Waist:: {format_si(Cs_power_per_trap_ideal)}W")
print(f"Cs Trap Depth/Temperaturew/ Diffraction Limited Beam Waist: : {format_si(Cs_U0_ideal)}K")

Cs Radial Trap Intensity: 19.9 GW/m^2 or 1.99 MW/cm^2
Cs Power Per Trap: 46 mW
Cs Trap Depth/Temperature: 5.22 mK

Cs Radial Trap Intensity w/ Diffraction Limited Beam Waist: 4.31 GW/m^2 or 431 W/cm^2
Cs Power Per Trap w/ Diffraction Limited Beam Waist:: 2.16 mW
Cs Trap Depth/Temperaturew/ Diffraction Limited Beam Waist: : 1.13 mK


In [19]:
print(f"Rb Radial Trap Intensity: {format_si(Rb_Io_radial)}W/m^2 or {format_si(Rb_Io_radial/1e4)}W/cm^2")
print(f"Rb Power Per Trap: {format_si(Rb_power_per_trap)}W")
print(f"Rb Trap Depth/Temperature: {format_si(Rb_U0)}K")
print("Note: Atom Array 2 measures 0.9 mK trap depths in tweezers")
print("")
print(f"Rb Radial Trap Intensity w/ Diffraction Limited Beam Waist: {format_si(Rb_Io_radial_ideal)}W/m^2 or {format_si(Rb_Io_radial_ideal/1e7)}W/cm^2")
print(f"Rb Power Per Trap w/ Diffraction Limited Beam Waist:: {format_si(Rb_power_per_trap_ideal)}W")
print(f"Rb Trap Depth/Temperaturew/ Diffraction Limited Beam Waist: : {format_si(Rb_U0_ideal)}K")

Rb Radial Trap Intensity: 607 MW/m^2 or 60.7 kW/cm^2
Rb Power Per Trap: 772 µW
Rb Trap Depth/Temperature: 814 µK
Note: Atom Array 2 measures 0.9 mK trap depths in tweezers

Rb Radial Trap Intensity w/ Diffraction Limited Beam Waist: 138 MW/m^2 or 13.8 W/cm^2
Rb Power Per Trap w/ Diffraction Limited Beam Waist:: 39.9 µW
Rb Trap Depth/Temperaturew/ Diffraction Limited Beam Waist: : 185 µK
